<a href="https://colab.research.google.com/github/jgermanob/JobOffersClassifier/blob/master/LSTM_trainable_split_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Acceso a drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Bibliotecas necesarias

In [2]:
!pip3 install pickle5

     |████████████████████████████████| 133kB 15.4MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219255 sha256=a1b526fc65dc30a60e23165b18ec394664f3ce58dff9f0d0f943c377f6738721
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from gensim.models import KeyedVectors
import tensorflow.keras.backend as K
import numpy as np
import io
import datetime
import pickle5 as pickle
from sklearn.metrics import classification_report

## Definición del modelo

In [4]:
class LSTM:
  def __init__(self, input_dim, embedding_matrix, input_length=300):
    output_dim = 300
    input = Input(shape=(input_length,), name='input_1')
    embeddings = layers.Embedding(input_dim+1,output_dim,input_length=input_length, weights=[embedding_matrix], trainable=True)
    lstm = layers.LSTM(100, name='lstm_layer')

    embedding_output = embeddings(input)

    embedding_output = layers.SpatialDropout1D(0.2)(embedding_output)

    lstm_output = lstm(embedding_output)
    
    predictions = layers.Dense(23, activation='softmax', name='dense_layer')(lstm_output)
    self.model = Model([input],predictions)
    self.__compile()
    #plot_model(self.model, to_file='/content/drive/My Drive/dicode/Backtranslation/lstm_model.png', show_shapes=True, show_dtype=True)
    print(self.model.summary())
  
  def __compile(self):
      self.model.compile(loss = tf.keras.losses.CategoricalCrossentropy() , optimizer='adam', metrics=['accuracy'])
    
  def fit(self, input, targets, epochs=5, batch_size=128):
      early_stopping_monitor = EarlyStopping(monitor = 'loss',patience = 20, min_delta=0.00001)
      callbacks = [early_stopping_monitor]
      history = self.model.fit([input], targets, epochs = epochs, batch_size = batch_size, callbacks = callbacks)
    
  def predict(self, input):
      return self.model.predict([input])
        
  def evaluate(self, input, targets, batch_size=128):
      return self.model.evaluate([input], targets, batch_size=batch_size)


## Entrenamiento y evaluación

In [5]:
def load_data(path):
  data = np.load(path)
  return data['arr_0']

In [6]:
x_train = load_data('/content/drive/My Drive/dicode/Backtranslation/Split/Train/x_train_base.npz')
y_train = load_data('/content/drive/My Drive/dicode/Backtranslation/Split/Train/y_train_base.npz')
print(x_train.shape)
print(y_train.shape)
x_test = load_data('/content/drive/My Drive/dicode/Backtranslation/Split/Test/x_test_base.npz')
y_test = load_data('/content/drive/My Drive/dicode/Backtranslation/Split/Test/y_test_base.npz')
embedding_matrix = load_data('/content/drive/My Drive/dicode/Backtranslation/Split/Train/embedding_matrix_train_base.npz')
with open('/content/drive/My Drive/dicode/Backtranslation/Split/Train/tokenizer_train_base.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

input_dim = len(tokenizer.word_index)
lstm_model = LSTM(input_dim=input_dim, embedding_matrix=embedding_matrix)
lstm_model.fit(x_train, y_train, epochs=10)
lstm_model.evaluate(x_test, y_test)
y_pred = lstm_model.predict(x_test)
y_predicted = np.argmax(y_pred, axis=-1)
y_true = np.argmax(y_test, axis=-1)
report = classification_report(y_true, y_predicted)
print(report)
output_file = open('/content/drive/My Drive/dicode/Backtranslation/Resultados/Split/classification_report_lstm_trainable_base.txt', 'w',encoding='utf8')
output_file.write(report)
output_file.close()



(48185, 300)
(48185, 23)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 300)          37010400  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 300)          0         
_________________________________________________________________
lstm_layer (LSTM)            (None, 100)               160400    
_________________________________________________________________
dense_layer (Dense)          (None, 23)                2323      
Total params: 37,173,123
Trainable params: 37,173,123
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
377/377 [==============================] - 166s 360ms/step - loss: 2.3507 